In [1]:
model_name='google/electra-base-discriminator'


#Data Collection

The data that i'm gonna use is the Pandora data set, 
https://paperswithcode.com/dataset/pandora. PANDORA is the first large-scale dataset of Reddit comments labeled with different personality models (including the well-established Big 5 model and MBTI model) 

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd 

The pandora dataset come in three different files

1.   author_profiles.csv
2.   all_comments_since_2015
3.   subreddits_mbti.csv





#The author_profiles.csv file: 
this file Contain more the 10K user profile where 9k are annotated with the MBTI personality test traits and 1k are annotated with the BIG5 traits.

In [4]:
df=pd.read_csv("./drive/MyDrive/Data/author_profiles.csv")

In [5]:
df.columns

Index(['author', 'mbti', 'introverted', 'intuitive', 'thinking', 'perceiving',
       'gender', 'age', 'enneagram', 'country', 'state', 'type',
       'agreeableness', 'openness', 'conscientiousness', 'extraversion',
       'neuroticism', 'is_description', 'is_percentile', 'is_score',
       'contains_details', 'num_comments', 'en_comments',
       'en_comments_percentage', 'region', 'continent', 'country_code',
       'enneagram_type', 'enneagram_wing', 'is_native_english_country',
       'predicted_test', 'test_name', 'test_scale', '16pers_ta',
       'test_result_type', 'is_female', 'is_female_pred', 'is_female_proba'],
      dtype='object')

The author profile contains differrent features suh as the author id , its  gender its age, its region , its country and the values for the personality traits.

We are interested only on the values for the MBTI and BIG5 personality traits:

'agreeableness', 'openness', 'conscientiousness', 'extraversion',
       'neuroticism',"introverted",	"intuitive",	"thinking",	"perceiving"

As in this work we aim to predict the personality of the user from its comments on the Reddit platform without taking into concideration its gender or age etc.. just from the text.


In [6]:
df_profile=df[["author",'agreeableness', 'openness', 'conscientiousness', 'extraversion',
       'neuroticism',"introverted",	"intuitive",	"thinking",	"perceiving"]]

In [7]:
len(df_profile.author.unique())

10295

In [8]:
#Verify the number of profiles that are annotqted with the Bi5 traits
df_profile[['agreeableness', 'openness', 'conscientiousness', 'extraversion','neuroticism']].dropna()

,agreeableness,openness,conscientiousness,extraversion,neuroticism
1,0.0,99.0,96.0,60.0,1.0
6,39.0,92.0,1.0,18.0,4.0
7,50.0,85.0,15.0,50.0,30.0
8,50.0,85.0,50.0,85.0,50.0
23,60.0,67.0,45.0,10.0,47.0
...,...,...,...,...,...
10290,0.0,92.0,52.0,1.0,79.0
10291,21.0,96.0,86.0,32.0,0.0
10292,84.0,78.0,56.0,0.0,2.0
10293,6.0,91.0,88.0,15.0,4.0


In [9]:
#Verify the number of profiles that are annotated with the MBTI traits
df_profile[["introverted",	"intuitive",	"thinking",	"perceiving"]].dropna()

,introverted,intuitive,thinking,perceiving
0,1.0,1.0,1.0,1.0
2,0.0,1.0,1.0,1.0
3,0.0,1.0,1.0,0.0
4,1.0,1.0,1.0,1.0
5,0.0,1.0,1.0,1.0
...,...,...,...,...
10289,0.0,1.0,0.0,1.0
10290,1.0,1.0,1.0,0.0
10291,1.0,1.0,1.0,0.0
10293,1.0,1.0,1.0,0.0


In [10]:
#Verify the number of profiles that are annotated with both BIG5 and  MBTI traits
df_profile[['agreeableness', 'openness', 'conscientiousness', 'extraversion',
       'neuroticism',"introverted",	"intuitive",	"thinking",	"perceiving"]].dropna()

,agreeableness,openness,conscientiousness,extraversion,neuroticism,introverted,intuitive,thinking,perceiving
7,50.0,85.0,15.0,50.0,30.0,0.0,1.0,1.0,1.0
23,60.0,67.0,45.0,10.0,47.0,1.0,1.0,0.0,1.0
58,0.0,37.0,72.0,72.0,3.0,0.0,0.0,1.0,1.0
138,10.0,80.0,74.0,27.0,18.0,1.0,1.0,1.0,0.0
169,1.0,50.0,61.0,90.0,3.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
10289,3.0,85.0,19.0,50.0,92.0,0.0,1.0,0.0,1.0
10290,0.0,92.0,52.0,1.0,79.0,1.0,1.0,1.0,0.0
10291,21.0,96.0,86.0,32.0,0.0,1.0,1.0,1.0,0.0
10293,6.0,91.0,88.0,15.0,4.0,1.0,1.0,1.0,0.0


we can see that in this dataset we have exactlu 10295 unique profile where 


1.   1568 are annotated with the BIG5 traits
2.   9067 are annotated with the MBTI traits
3.   377  are annotated with both MBTI and BIG5 traits


As in this work we aim to predict both the MBTI and BIG5 traits, we will focus only on the 377 profiles that are annotated with both of them.

In [11]:
df_profile=df_profile.dropna(subset=['introverted',	'intuitive',	'thinking',	'perceiving','agreeableness',	'openness',	'conscientiousness',	'extraversion',	'neuroticism'])
df_profile

,author,agreeableness,openness,conscientiousness,extraversion,neuroticism,introverted,intuitive,thinking,perceiving
7,-BlitzN9ne,50.0,85.0,15.0,50.0,30.0,0.0,1.0,1.0,1.0
23,-dyad-,60.0,67.0,45.0,10.0,47.0,1.0,1.0,0.0,1.0
58,12345jk12345,0.0,37.0,72.0,72.0,3.0,0.0,0.0,1.0,1.0
138,ACE_C0ND0R,10.0,80.0,74.0,27.0,18.0,1.0,1.0,1.0,0.0
169,A_Bra_and_a_Ham,1.0,50.0,61.0,90.0,3.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
10289,quakeroaks,3.0,85.0,19.0,50.0,92.0,0.0,1.0,0.0,1.0
10290,rrgjl,0.0,92.0,52.0,1.0,79.0,1.0,1.0,1.0,0.0
10291,seldomvanilla,21.0,96.0,86.0,32.0,0.0,1.0,1.0,1.0,0.0
10293,turncloak471,6.0,91.0,88.0,15.0,4.0,1.0,1.0,1.0,0.0


As we can see in the previous table , the MBTI trait values {'introverted',	'intuitive',	'thinking',	'perceiving'} are binary values (they contain only 0 or 1 to define whether the profile belong to the category or not). However for the BIG5 traits, we can see that they are numerical values (for 0 to 100) which instead of indicating wether the profile belong to a category or not like the MBTI does, these values indicates to which degree a profile belong to a category. meaning that in the BIG5 traits a profile will belong to all the categories but with a degree of beloniging. Wheras in MBTI the values defines i f the user belong to the trait or not.

### Contribution
From the table above we can see that we have two problems:

1.   First we have a multi label classification task for the MBTI traits
2.   First we have a multi label regression task for the BIG5 traits

In order to solve thease two problems we are proposing a Multi-Head (classification,regression) network that given a text it will predict at the same time both the value of each MBTI an Big5 trait factor.


Now that we filtred the profiles that we arw gonna use, we will combine them wiith all their comments that they said since 2015.

To do that we need to call the all all_comments_since_2015.csv that comes with the Pendora package

In [12]:
df=pd.read_csv("./drive/MyDrive/Data/all_comments_since_2015.csv")

In [13]:
len(df)


17640062

#### all_comments_since_2015.csv

This file contain all the comments from 2015 to 2020 for all the profiles in the Pnadora dataset. It contains more than 17M comment.
However due to the fact not all profiles are annotated with both MBTI and BIG5 factors. We will keep only the comments that correspond to the filtred profiles

In [14]:
df.columns

Index(['author', 'author_flair_text', 'body', 'downs', 'created_utc',
       'subreddit_id', 'link_id', 'parent_id', 'score', 'controversiality',
       'gilded', 'id', 'subreddit', 'ups', 'word_count',
       'word_count_quoteless', 'lang'],
      dtype='object')

In [15]:
df

,author,author_flair_text,body,downs,created_utc,subreddit_id,link_id,parent_id,score,controversiality,gilded,id,subreddit,ups,word_count,word_count_quoteless,lang
0,MetricExpansion,ENTP,Those stats come from the test. [Echoing the c...,0.0,1.474429e+09,t5_2s90r,t3_53plrw,t3_53plrw,6.0,0.0,0.0,d7vkyrf,mbti,6.0,151.0,149.0,en
1,MetricExpansion,NaN,"That's great to hear! I hope you know that, de...",0.0,1.480139e+09,t5_2s90r,t3_5ep948,t1_dafz6ab,1.0,0.0,0.0,dafzzrg,mbti,0.0,319.0,316.0,en
2,MetricExpansion,[ENTP-5 M 22],I can totally agree on reticence! With respect...,0.0,1.455096e+09,t5_2s90r,t3_44q2vf,t1_cztchk3,1.0,0.0,0.0,czul5ag,mbti,1.0,145.0,143.0,en
3,MetricExpansion,<U+1D07><U+0274><U+1D1B><U+1D18> - <U+1D1B><U+...,I took it several times. I'm typed as TYPE_MEN...,0.0,1.462865e+09,t5_2s90r,t3_4ijf4l,t3_4ijf4l,1.0,0.0,0.0,d2zo611,mbti,1.0,41.0,41.0,en
4,MetricExpansion,<U+1D07><U+0274><U+1D1B><U+1D18> - <U+1D1B><U+...,Gawd it's like we don't even need drugs to be ...,0.0,1.460656e+09,t5_2s90r,t3_4eptxr,t1_d22uh4r,1.0,0.0,0.0,d22uu81,mbti,1.0,11.0,11.0,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17640057,ilikehockeyandguitar,NaN,I kinda feel the same way about Animal Collect...,NaN,1.527879e+09,t5_2zj24,t3_8nt3u5,t1_dzy8o1n,3.0,0.0,0.0,dzyenpb,indieheads,NaN,38.0,38.0,en
17640058,Burning_Lovers,:flag-ca: California,have you seen how delusional Republican voters...,NaN,1.541518e+09,t5_2cneq,t3_9unf4i,t1_e95v5f3,2.0,0.0,0.0,e95vz50,politics,NaN,20.0,19.0,en
17640059,WinterCharm,NaN,Good stuff :D I love immunology.,NaN,1.523051e+09,t5_2szyo,t3_8ab9lo,t1_dwxqjef,2.0,0.0,0.0,dwxqo62,Showerthoughts,NaN,6.0,6.0,en
17640060,MizzElissa,��,You're not my real mom,NaN,1.530637e+09,t5_2vegg,t3_8vsap7,t1_e1q4vxt,15.0,0.0,0.0,e1q73lg,me_irl,NaN,5.0,5.0,en


The dataset contains diffrent features suche as the wor_count of the comment, the time of creating the comment, language, the content of the comment, the author id etc..
In this work we focus only on the content of the comment, as we are trying to predict the personality from the content of the user comments.

In [16]:
#we zill keep only the "author",'body' and "created_utc"
df_comments=df[["author",'body',"created_utc"]]

an example of a one comment of a profile

In [17]:

df_comments["body"][0]

"Those stats come from the test. [Echoing the comment I just made on a related question](https://www.reddit.com/r/mbti/comments/53q3qt/why_do_so_many_people_ask_to_be_typed_when_you/d7vf1im), it depends on what an MBTI type means to you.If it's the four letter code from the dichotomies, then those type frequencies are legitimate as are the other scientific results where test-derived MBTI types are studied, up to the quality of the scientific work, of course. But then we're not guaranteed to be talking about cognitive functions at all.If it's the set of cognitive functions you use with a certain priority, as many here prefer to talk about, then those statistics and any scientific results pertaining to MBTI types are just *completely and utterly irrelevant* to the discussion because you're just not talking about the same thing they are. This includes TYPE_MENTION being rare, TYPE_MENTION having the highest IQ, TYPE_MENTION making the most money, TYPE_MENTION being the most common type in

#### Concatenate the filtered profiles with their comments.

In [18]:
alld_data=pd.merge(df_comments,df_profile, on='author')

In [19]:
len(alld_data)

1045374

by concatinating the filtred profiles with their comments we went down to 1M

It is true that loosing rows of data is not preferable. However we needed to do that because the other rows refer to profiles that either have MBTI traits or BIG5 traits not both. Also 1M row of data is still a huge data quantity for a network to be trained on. 

In [20]:
alld_data.dropna(inplace=True)

In [21]:
alld_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1045374 entries, 0 to 1045373
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   author             1045374 non-null  object 
 1   body               1045374 non-null  object 
 2   created_utc        1045374 non-null  float64
 3   agreeableness      1045374 non-null  float64
 4   openness           1045374 non-null  float64
 5   conscientiousness  1045374 non-null  float64
 6   extraversion       1045374 non-null  float64
 7   neuroticism        1045374 non-null  float64
 8   introverted        1045374 non-null  float64
 9   intuitive          1045374 non-null  float64
 10  thinking           1045374 non-null  float64
 11  perceiving         1045374 non-null  float64
dtypes: float64(10), object(2)
memory usage: 103.7+ MB


Now lets take a look on the final data that we have

In [22]:
alld_data

,author,body,created_utc,agreeableness,openness,conscientiousness,extraversion,neuroticism,introverted,intuitive,thinking,perceiving
0,MetricExpansion,Those stats come from the test. [Echoing the c...,1.474429e+09,30.0,70.0,15.0,15.0,50.0,0.0,1.0,1.0,1.0
1,MetricExpansion,"That's great to hear! I hope you know that, de...",1.480139e+09,30.0,70.0,15.0,15.0,50.0,0.0,1.0,1.0,1.0
2,MetricExpansion,I can totally agree on reticence! With respect...,1.455096e+09,30.0,70.0,15.0,15.0,50.0,0.0,1.0,1.0,1.0
3,MetricExpansion,I took it several times. I'm typed as TYPE_MEN...,1.462865e+09,30.0,70.0,15.0,15.0,50.0,0.0,1.0,1.0,1.0
4,MetricExpansion,Gawd it's like we don't even need drugs to be ...,1.460656e+09,30.0,70.0,15.0,15.0,50.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1045369,Mkmcat,"Actually, I double checked my MBTI and I am EN...",1.519730e+09,0.0,98.0,83.0,98.0,6.0,0.0,1.0,1.0,0.0
1045370,Mkmcat,"Yes, my ring fingers are on both hands longer ...",1.519128e+09,0.0,98.0,83.0,98.0,6.0,0.0,1.0,1.0,0.0
1045371,NinjaMine,Summoner name: NinjaMineI'm level 4 Teemo tras...,1.496300e+09,50.0,54.0,72.0,19.0,84.0,1.0,1.0,0.0,0.0
1045372,NinjaMine,Openness - 77%Conscientiousness - 68%Extravers...,1.521167e+09,50.0,54.0,72.0,19.0,84.0,1.0,1.0,0.0,0.0


# Model creation

Now that we filtred and created the data that we need, we will start now by creating the learning model architecture.

To do that in a nutshell we will use the hugging face librarie to fune-tune a pretrained model so that it encodes the comments of the profile in a contextual way. Then using thease vector embeddings we will train a multi-task neural network to predict both MBTI and BIG5 values. All of this is done in an END-To-END network.

We will start firs by importing all the necessary libraries

In [23]:

 
!pip install transformers


     |████████████████████████████████| 2.9 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 56 kB 4.6 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
     |████████████████████████████████| 3.3 MB 51.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [24]:
!pip install audtorch

     |████████████████████████████████| 54 kB 2.0 MB/s 


In [25]:
!nvidia-smi


Tue Oct 19 21:06:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
import os
import math
import audtorch
import torch
from torch.nn import BCEWithLogitsLoss,MSELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, ElectraTokenizer, ElectraModel, ElectraConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
%matplotlib inline

SoX could not be found!

    If you do not have SoX, proceed here:
     - - - http://sox.sourceforge.net/ - - -

    If you do (or think that you should) have SoX, double-check your
    path variables.
    


Before start creating the model we need first to prepare our data so that thei'll bee appropriate to be consumed by our model. 


We need first to devide our data into training, test and validation sets.


1.   The train data will be used as the main data to train the model
2.   the validation data will be used also during the training phase but just to validate the performance of the model during the training phase.
3.   The test data will be used as a holdout data that has not been seen by the model in order to truly evaluate the generalization of our model

We will use 669K of dqtq rows to train the model, 167k as a validation data and 209k as test data


In [27]:
from sklearn.model_selection import train_test_split
train,test = train_test_split(alld_data, test_size=0.2,random_state =0)
train,valid = train_test_split(train, test_size=0.2,random_state =0)

In [28]:
len(train)

669039

In [29]:
len(test)

209075

In [30]:
train=train[:int(len(train)/10)]

In [31]:
test=test[:int(len(test)/10)]

In [32]:
valid=valid[:int(len(valid)/10)]

In [33]:
len(train)

66903

In [34]:
len(test)

20907

Now that we have the data for the different tasks that we need. we will use a pretrained Electra transformer model to encode the context of the comments for each row. to do that we need first to instanciate the Electra pretrained model fron the Hugging face library 

In [35]:
#Define the Electra model name from the hugging face library
model_name='google/electra-small-discriminator'


Instanciate the tokenizer of the pre_trqined model

The tokenizer will transform th textual data into different vectors, where each vector present differnt isights such as the vector of positions of eqch word in the sentence, The vector of token repesentensqtion for eqch word in the sentence and the vector of masking values for each word in the sentence

In [36]:
#
tokenizer = ElectraTokenizer.from_pretrained(model_name, do_lower_case=True)

train_data= train["body"].values
test_data = test["body"].values
valid_data = valid["body"].values



Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In order to gather the ensights that we need we define a helper function 
Input_Tokenization that will instanciate the pre-trained Electra tokenizer and tokenize each row sentence. We will apply this function for all the training, test, and validation data so that we create mebedding vectors for each row on them. Thoes embedded vectors will be passed then to the Prediction network


In [37]:
def Input_Tokenization(liste, tokenizer):
    
    outputs = tokenizer.batch_encode_plus(liste,padding=True,truncation=True,return_tensors='pt',return_attention_mask=True)    

    return (outputs.input_ids,outputs.attention_mask)

In [38]:
train_input_ids,train_attention_masks=Input_Tokenization(train_data, tokenizer)

In [39]:
test_input_ids,test_attention_masks=Input_Tokenization(test_data, tokenizer)

In [40]:
valid_input_ids,valid_attention_masks=Input_Tokenization(valid_data, tokenizer)

If we take a closer look to the train_input_ids wich represent the encoding of each row in the training data we can see that the row of texts were transformed into rows of vector of numbers, where each vector of numbers represent a textual comment and each number in the vector represent a word identificator in the comment. The numbers are token representation were each number refer to a word in the pre_trained Electra vocabulary

In [41]:
train_input_ids

tensor([[ 101, 2009, 1005,  ...,    0,    0,    0],
        [ 101, 2327, 9874,  ...,    0,    0,    0],
        [ 101, 2619, 2425,  ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2245,  ...,    0,    0,    0],
        [ 101, 7110, 1005,  ...,    0,    0,    0],
        [ 101, 2023, 2003,  ...,    0,    0,    0]])

In [42]:
train_input_ids[0]

tensor([  101,  2009,  1005,  1055,  2069,  2026,  6772,  2008,  2045,  2003,
         2070,  4066,  1997,  5415,  2344,  1998,  3800,  2008,  7906,  2033,
         2013,  2108,  1037, 13076,  9152, 19466,  2923,  1012,  2026,  2828,
         1035,  5254,  2190,  2767,  1998,  1045,  2024,  1031,  2183,  2004,
         1996,  3124,  2006,  1996,  2157,  2005, 14414,  1033,  1006, 16770,
         1024,  1013,  1013, 27263,  2015,  1012,  2006,  5332, 17644,  1012,
         4012,  1013, 12991, 13874,  1011,  2498,  1011,  1999,  1011,  2166,
         1011, 14085,  7747,  1011,  9152, 19466,  2964,  1011,  4507,  1011,
        19734, 12190,  1011,  4090,  2683, 28311, 17134,  1012,  1052,  3070,
         1007,  1010,  2138,  2057,  2119,  6709,  2007,  2009,  2061,  2172,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

Now that we transformed our data into a numerical values 'vectors of numbers' instead of the original textual values we will now transform them into a convininet form to be consumed by the Pytorch platform

To do that we need first to define the names of label features

In [43]:
label_cols = ['agreeableness', 'openness',
       'conscientiousness', 'extraversion', 'neuroticism', 'introverted',
       'intuitive', 'thinking', 'perceiving']

Like we said, our model is a multi task prediction model. so we have classification labels and regression labels.

In [44]:
#Classification labels
clas_label_cols=[ 'introverted',
       'intuitive', 'thinking', 'perceiving']

In [45]:
#Regression labels
reg_label_cols=['agreeableness', 'openness',
       'conscientiousness', 'extraversion', 'neuroticism']

During the training phase we will need for both the training set and the validation set. Meaning that we will need for both the training and validation vector representations as well as the the classification and regression labels.

For each row we need to  have the tokenization(ids and mask) of that row and ist MBTI(classification labels) and BIG5(regression labels) values.

In [46]:


X_train,train_masks,Y_train_r,Y_train_c = (train_input_ids,train_attention_masks,torch.tensor(train[reg_label_cols].values.tolist(),dtype=torch.float32),torch.tensor(train[clas_label_cols].values.tolist(),dtype=torch.float32))
X_valid,valid_masks,Y_valid_r,Y_valid_c = (valid_input_ids,valid_attention_masks,torch.tensor(valid[reg_label_cols].values.tolist(),dtype=torch.float32),torch.tensor(valid[clas_label_cols].values.tolist(),dtype=torch.float32))


if we take a closer look to the first row information in the training data we can find :

In [47]:
X_train[0]# the comment representation 'tokenization' of the first row
#We can see a lot of 0 values. This is because the vector size of the pre-trqined Electra model id 256. and the nuber of words in the first comment are less than 256 
#So the 0 values are added as a padding values to make all the vectors have the same size

tensor([  101,  2009,  1005,  1055,  2069,  2026,  6772,  2008,  2045,  2003,
         2070,  4066,  1997,  5415,  2344,  1998,  3800,  2008,  7906,  2033,
         2013,  2108,  1037, 13076,  9152, 19466,  2923,  1012,  2026,  2828,
         1035,  5254,  2190,  2767,  1998,  1045,  2024,  1031,  2183,  2004,
         1996,  3124,  2006,  1996,  2157,  2005, 14414,  1033,  1006, 16770,
         1024,  1013,  1013, 27263,  2015,  1012,  2006,  5332, 17644,  1012,
         4012,  1013, 12991, 13874,  1011,  2498,  1011,  1999,  1011,  2166,
         1011, 14085,  7747,  1011,  9152, 19466,  2964,  1011,  4507,  1011,
        19734, 12190,  1011,  4090,  2683, 28311, 17134,  1012,  1052,  3070,
         1007,  1010,  2138,  2057,  2119,  6709,  2007,  2009,  2061,  2172,
         1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [48]:
train_masks[0]# The mask represent binary values 0 or 1
#1 means that the nodel will take into concideration the word in that position while 0 means that the model
#will not take into concideration that word.
#This is logic: as we can see all the 0 are puted in the position of the padding values in the above 'x_train[0]' comments.
#which mean that the model will not concentrate on the padding values. It will concentrate only on the comment words values

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [49]:
Y_train_c[0]
# The classification labels 'MBTI labels' for the first comment are 

tensor([1., 1., 0., 0.])

In [50]:
Y_train_r[0]
# The regression labels 'Big5 labels' for the first comment are 

tensor([83., 80., 46., 37., 14.])

Now that we know how our data looks like and the meaning of each data representation we will transform the data in a Pytorch format usign the data loader function to make them in chunks in order to trqin our model in a batch way

In [51]:
batch_size = 15


train_data = TensorDataset(X_train, train_masks, Y_train_r,Y_train_c)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data,\
                              sampler=train_sampler,\
                              batch_size=batch_size)

validation_data = TensorDataset(X_valid, valid_masks, Y_valid_r,Y_valid_c)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data,\
                                   sampler=validation_sampler,\
                                   batch_size=batch_size)

In [52]:
from   torch.nn import BCELoss

Now that the data is ready we will create the END-2-END neural network

In [53]:
'''class ElectraSharedWeights_Multi_Task(torch.nn.Module):
  
  def __init__(self, r_num_labels=2,c_num_labels=2):
    super(ElectraForMultiLabelSequenceClassification, self).__init__()
    self.c_num_labels = c_num_labels#The number of the classification labels whiche are 4 labels to use them as the final size for  the final linear layer
    self.r_num_labels = r_num_labels#The number of the regression labels whiche are 5 labelsto use them as the final size for  the final linear layer
    self.electra = ElectraModel.from_pretrained(model_name)#Instanciate  the pretrained model
    self.layer1 = torch.nn.Linear(256, 150)# add The first linear layer on top of the encoding  discriminator layers of Electra 
    self.layer2  = torch.nn.Linear(150, 100)# add the second linear layer on top of the first one
    self.layer3  = torch.nn.Linear(100, 50)# add the third linear layer on top of the second one
    self.normilizer1  = torch.nn.BatchNorm1d(256)# add the normalizer to the electra values
    self.normilizer2  = torch.nn.BatchNorm1d(100)# add the  normilizer layer onto the second layer values

    #All the previous layers will be shared by the last separate two layers (classifier layer and regressor layer)

    self.classifier = torch.nn.Linear(50, c_num_labels)# add the final linear layer for classification on top of the third layer   
    self.regressor = torch.nn.Linear(50, r_num_labels)# add the final linear layer for regression on top of the third layer 
    self.drop=torch.nn.Dropout(p=0.4)# Define a dropout layer to overpass the overfitting during the training phase
    self.leaky=torch.nn.LeakyReLU()# Use the Leaky Relu activation function as the ;ain activation function for the shared layers



    #In order to train our models the weights need to be initialized, because if they were at 0, then the model will not be optimized so we need to
    # inilize them. The most popular way to inisialize the weights is to use the Xavier initializer
    torch.nn.init.xavier_normal_(self.layer1.weight) #initialise the weight of the first linear layer
    torch.nn.init.xavier_normal_(self.layer2.weight) #initialise the weight of the second linear layer
    torch.nn.init.xavier_normal_(self.layer3.weight) #initialise the weight of the third linear layer
    torch.nn.init.xavier_normal_(self.classifier.weight) #initialise the weight of the classification layer
    torch.nn.init.xavier_normal_(self.regressor.weight) #initialise the weight of th regression layer

  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels_r=None,labels_c=None):
    #Now that we defined the architecture and initialized the weights we need to define the forward function
    #and how each defined variable will be used in the architecture

    # last hidden layer
    #for ech row we will use the ELECTRa model to create an embedding vector given the tokenized input
    contextual_embedding = self.electra(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids)

    # pool the outputs into a mean vector
    contextual_embedding = self.pool_hidden_state(contextual_embedding)
    cl1 = self.leaky(self.layer1(contextual_embedding))
    cl1=self.drop(cl1)
    cl2 = self.leaky(self.layer2(cl1))
    cl3 = self.leaky(self.layer3(cl2))
    cl3=self.drop(cl3)
    clogits = torch.sigmoid(self.classifier(cl3))

    rl1 = self.leaky(self.layer1(mean_last_hidden_state))
    rl1=self.drop(rl1)
    rl2 = self.leaky(self.layer2(rl1))
    rl3 = self.leaky(self.layer3(rl2))
    rl3=self.drop(rl3)
    rlogits = self.regressor(rl3)
        
    if ((labels_c is not None) and(labels_r is not None)):
      closs_fct =BCEWithLogitsLoss()
      rloss_fct = MSELoss()
      #rloss_fct = audtorch.metrics.functional.pearsonr

      closs = closs_fct(clogits.view(-1, self.c_num_labels),\
                      labels_c.view(-1, self.c_num_labels))
      rloss=rloss_fct(rlogits.view(-1, self.r_num_labels),\
                      labels_r.view(-1, self.r_num_labels))

      loss=(rloss+closs)/2
      return loss
    else:
      return clogits,rlogits


  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state
    
  

# len(Y_train[0]) = 6
'''

'class ElectraSharedWeights_Multi_Task(torch.nn.Module):\n  \n  def __init__(self, r_num_labels=2,c_num_labels=2):\n    super(ElectraForMultiLabelSequenceClassification, self).__init__()\n    self.c_num_labels = c_num_labels#The number of the classification labels whiche are 4 labels to use them as the final size for  the final linear layer\n    self.r_num_labels = r_num_labels#The number of the regression labels whiche are 5 labelsto use them as the final size for  the final linear layer\n    self.electra = ElectraModel.from_pretrained(model_name)#Instanciate  the pretrained model\n    self.layer1 = torch.nn.Linear(256, 150)# add The first linear layer on top of the encoding  discriminator layers of Electra \n    self.layer2  = torch.nn.Linear(150, 100)# add the second linear layer on top of the first one\n    self.layer3  = torch.nn.Linear(100, 50)# add the third linear layer on top of the second one\n    self.normilizer1  = torch.nn.BatchNorm1d(256)# add the normalizer to the electr

In [72]:
class ElectraSharedWeights_Multi_Task(torch.nn.Module):
  
  def __init__(self, r_num_labels=2,c_num_labels=2):
    super(ElectraSharedWeights_Multi_Task, self).__init__()
    self.c_num_labels = c_num_labels#The number of the classification labels whiche are 4 labels to use them as the final size for  the final linear layer
    self.r_num_labels = r_num_labels#The number of the regression labels whiche are 5 labelsto use them as the final size for  the final linear layer
    self.electra = ElectraModel.from_pretrained(model_name)#Instanciate  the pretrained model
    self.layer1 = torch.nn.Linear(256, 150)# add The first linear layer on top of the encoding  discriminator layers of Electra 
    self.layer2  = torch.nn.Linear(150, 100)# add the second linear layer on top of the first one
    self.layer3  = torch.nn.Linear(100, 50)# add the third linear layer on top of the second one
    #self.normilizer1  = torch.nn.BatchNorm1d(256)# add the normalizer to the electra values
    #self.normilizer2  = torch.nn.BatchNorm1d(100)# add the  normilizer layer onto the second layer values
    #All the previous layers will be shared by the last separate two layers (classifier layer and regressor layer)

    self.classifier = torch.nn.Linear(50, c_num_labels)# add the final linear layer for classification on top of the third layer   
    self.regressor = torch.nn.Linear(50, r_num_labels)# add the final linear layer for regression on top of the third layer 
    self.drop=torch.nn.Dropout(p=0.4)# Define a dropout layer to overpass the overfitting during the training phase
    self.leaky=torch.nn.LeakyReLU()# Use the Leaky Relu activation function as the ;ain activation function for the shared layers



    #In order to train our models the weights need to be initialized, because if they were at 0, then the model will not be optimized so we need to
    # inilize them. The most popular way to inisialize the weights is to use the Xavier initializer
    torch.nn.init.xavier_normal_(self.layer1.weight) #initialise the weight of the first linear layer
    torch.nn.init.xavier_normal_(self.layer2.weight) #initialise the weight of the second linear layer
    torch.nn.init.xavier_normal_(self.layer3.weight) #initialise the weight of the third linear layer
    torch.nn.init.xavier_normal_(self.classifier.weight) #initialise the weight of the classification layer
    torch.nn.init.xavier_normal_(self.regressor.weight) #initialise the weight of th regression layer

  def forward(self, input_ids, token_type_ids=None,\
              attention_mask=None, labels_r=None,labels_c=None):
    #Now that we defined the architecture and initialized the weights we need to define the forward function
    #and how each defined variable will be used in the architecture

    # last hidden layer
    #for ech row we will use the ELECTRa model to create an embedding vector given the tokenized input
    Electra_hidding_state = self.electra(input_ids=input_ids,\
                                   attention_mask=attention_mask,\
                                   token_type_ids=token_type_ids)
    contextual_embedding = Electra_hidding_state[0]
    contextual_embedding = torch.mean(contextual_embedding, 1)
    #contextual_embedding=self.normilizer1(contextual_embedding)
    # pool the outputs into a mean vector
    cl1 = self.leaky(self.layer1(contextual_embedding))
    cl1=self.drop(cl1)
    cl2 = self.leaky(self.layer2(cl1))
    #cl2 = self.normilizer2(cl2)
    cl3 = self.leaky(self.layer3(cl2))
    cl3=self.drop(cl3)

    #All the previous weights will be shared by both last layers (classifier and regresor) 
    clogits = self.classifier(cl3)# The classification results(MBTI Labels) given the shared weights

    rlogits = self.regressor(cl3)# The regression results(BIG5 labels) given the shared weights


    # Due to the fact that we have a multi-task learning model then we will have two different losses
    # one to compute the classification loss and the other is to compute the regression loss
    # We will use the BCE version for the classification loss and the MSE for the regression loss
        
    closs_fct =BCEWithLogitsLoss()# Classification loss
    rloss_fct = MSELoss()# Regression loss

    if ((labels_r is not None) and (labels_c is not None)):

  
      closs = closs_fct(clogits.view(-1, self.c_num_labels),\
                        labels_c.view(-1, self.c_num_labels))
      rloss=rloss_fct(rlogits.view(-1, self.r_num_labels),\
                        labels_r.view(-1, self.r_num_labels))
      #Because a model need to have one loss function to be trained on and be cause we need to minimise both regression and classification loss
      #we create a combination of the regression loss and the classification loss by summing them and deviding by 2
      #By combining both losses  into a one loss we will force the model to minimize the combination loss wich will lead to the minimization of the 
      #regression and the classification loss (minimizing their sum will lead to minimizing their original values)
      loss=(rloss+closs)/2
      return loss
    else:
      return clogits,rlogits



Noow that we created the architecture of our model we will instanciate it, make it capable of runing in paralell on defining it to be running on GPUs

In [73]:
model = ElectraSharedWeights_Multi_Task(c_num_labels=len(Y_train_c[0]),r_num_labels=len(Y_train_r[0]))
model = torch.nn.DataParallel(model)
model.cuda()


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DataParallel(
  (module): ElectraSharedWeights_Multi_Task(
    (electra): ElectraModel(
      (embeddings): ElectraEmbeddings(
        (word_embeddings): Embedding(30522, 128, padding_idx=0)
        (position_embeddings): Embedding(512, 128)
        (token_type_embeddings): Embedding(2, 128)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
      (encoder): ElectraEncoder(
        (layer): ModuleList(
          (0): ElectraLayer(
            (attention): ElectraAttention(
              (self): ElectraSelfAttention(
                (query): Linear(in_features=256, out_features=256, bias=True)
                (key): Linear(in_features=256, out_features=256, bias=True)
                (value): Linear(in_features=256, out_features=256, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
           

In [63]:
!pip install hiddenlayer 

In [64]:

import torchvision.models
import hiddenlayer as hl

We also need to define the Optimizer that we will use during the learning process for the gradient propagation. We will use the popular Adam optimizer with a 0.01 weight decay and a 2e-5 learning rate

In [65]:
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01, correct_bias=False)


Now we will define the training function for our model

In [66]:
def train(model, num_epochs,\
          optimizer,\
          train_dataloader, valid_dataloader,\
          model_save_path,\
          train_loss_set=[], valid_loss_set = [],\
          lowest_eval_loss=None, start_epoch=0,\
          device="cpu"
          ):
  """
  Train the model and save the model with the lowest validation loss
  """

  model.to(device)

  for i in trange(num_epochs, desc="Epoch"):
    
    actual_epoch = start_epoch + i

    

    model.train()

    tr_loss = 0
    num_train_samples = 0

    for step, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_r,b_labels_c = batch
      # Clear out the gradients (by default they accumulate)
      optimizer.zero_grad()
      # Forward pass
      loss = model(b_input_ids, attention_mask=b_input_mask, labels_r=b_labels_r,labels_c=b_labels_c)
      # store train loss
      tr_loss += loss.item()
      num_train_samples += b_labels_c.size(0)
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      #scheduler.step()

    # Update tracking variables
    epoch_train_loss = tr_loss/num_train_samples
    train_loss_set.append(epoch_train_loss)

    print("Train loss: {}".format(epoch_train_loss))

    # Validation

    # Put model in evaluation mode to evaluate loss on the validation set
    model.eval()

    # Tracking variables 
    eval_loss = 0
    num_eval_samples = 0

    # Evaluate data for one epoch
    for batch in valid_dataloader:
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels_r,b_labels_c = batch
      # Telling the model not to compute or store gradients,
      # saving memory and speeding up validation
      with torch.no_grad():
        # Forward pass, calculate validation loss
        loss = model(b_input_ids, attention_mask=b_input_mask, labels_r=b_labels_r,labels_c=b_labels_c)
        eval_loss += loss.item()
        num_eval_samples += b_labels_c.size(0)

    epoch_eval_loss = eval_loss/num_eval_samples
    valid_loss_set.append(epoch_eval_loss)

    print("Valid loss: {}".format(epoch_eval_loss))

    if lowest_eval_loss == None:
      lowest_eval_loss = epoch_eval_loss
      # save model
      save_model(model, model_save_path, actual_epoch,\
                 lowest_eval_loss, train_loss_set, valid_loss_set)
    else:
      if epoch_eval_loss < lowest_eval_loss:
        lowest_eval_loss = epoch_eval_loss
        # save model
        save_model(model, model_save_path, actual_epoch,\
                   lowest_eval_loss, train_loss_set, valid_loss_set)
    print("\n")

  return model, train_loss_set, valid_loss_set


def save_model(model, save_path, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist):
  """
  Save the model to the path directory provided
  """
  model_to_save = model.module if hasattr(model, 'module') else model
  checkpoint = {'epochs': epochs, \
                'lowest_eval_loss': lowest_eval_loss,\
                'state_dict': model_to_save.state_dict(),\
                'train_loss_hist': train_loss_hist,\
                'valid_loss_hist': valid_loss_hist
               }
  torch.save(checkpoint, save_path)
  print("Saving model at epoch {} with validation loss of {}".format(epochs,\
                                                                     lowest_eval_loss))
  return
  
def load_model(save_path):
  """
  Load the model from the path directory provided
  """
  checkpoint = torch.load(save_path)
  model_state_dict = checkpoint['state_dict']
  model = ElectraForMultiLabelSequenceClassification(num_labels=model_state_dict["classifier.weight"].size()[0])
  model.load_state_dict(model_state_dict)

  epochs = checkpoint["epochs"]
  lowest_eval_loss = checkpoint["lowest_eval_loss"]
  train_loss_hist = checkpoint["train_loss_hist"]
  valid_loss_hist = checkpoint["valid_loss_hist"]
  
  return model, epochs, lowest_eval_loss, train_loss_hist, valid_loss_hist

Now we need to define for how many epochs our model will be runing and were we store the best model

In [67]:
num_epochs=20

cwd = os.getcwd()
model_save_path = output_model_file = os.path.join(cwd, "drive/MyDrive/SeedDoubleHead_OmlyElectraSharedWeights_20epochs_Electra.bin")


mow we will just train the model

In [55]:
model, train_loss_set, valid_loss_set = train(model=model,\
                                              num_epochs=num_epochs,\
                                              optimizer=optimizer,\
                                              train_dataloader=train_dataloader,\
                                              valid_dataloader=validation_dataloader,\
                                              model_save_path=model_save_path,\
                                              device="cuda")

Now that our model has been trained we will load the best version of the model that acheived the lowes loss on the validation set during the data and use it to predict the test data

In [74]:
checkpoint = torch.load(model_save_path)
model_state_dict = checkpoint['state_dict']
model = ElectraSharedWeights_Multi_Task(c_num_labels=4,r_num_labels=5)
model.load_state_dict(model_state_dict)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

To generate predictions from the results provided by our model for both regression and classification task we define a Predict function that gives us both predictions

In [75]:
def Predict(model, features,mask, num_label_c,num_label_r, device="cpu", batch_size=32):
  num_iter = math.ceil(features.shape[0]/batch_size)# define the number of iteration in all the data by batches
  model.eval() #Indicate that we will use the model to evaluation purposes

  c_pred_probs = np.array([]).reshape(0, num_label_c)# Create dummpy classification predictions 'all 0' vectors that we will modify
  r_pred_probs = np.array([]).reshape(0, num_label_r)# Create dummpy regression predictions 'all 0' vectors that we will modify
  model.to(device)
  d={}
  
  for i in range(num_iter):# for each batch
    X = features[i*batch_size:(i+1)*batch_size,:]#Get the features of the passed data
    masks = mask[i*batch_size:(i+1)*batch_size,:]#Get the masks of the passed data
    X = X.to(device)#transfer them to the device 'exmp cuda'
    masks = masks.to(device)#transfer them to the device 'exmp cuda'
    with torch.no_grad():
      logits_c,logits_r = model(input_ids=X, attention_mask=masks) # predict the values of the passed batch of data
      #logits = logits.sigmoid().detach().cpu().numpy()
      logits_c = logits_c.sigmoid().detach().cpu().numpy()# for the classification head we will aplly a sigmoid function as like we said the classification labels are binary labels
      c_pred_probs = np.vstack([c_pred_probs, logits_c])# stack them to the pr-defined dummy classification variable
      logits_r = logits_r.detach().cpu().numpy()# For the regression values wi well use directly the linear layer output because the regression labels are real values
      r_pred_probs = np.vstack([r_pred_probs, logits_r])# stack them to the pr-defined dummy regression variable
  d['class_pred']=np.round(c_pred_probs) # to get binary values for the sigmoids probability values we will use the .round() function for the classification results
  d['reg_pred']=r_pred_probs
  
  return d
  


In [76]:
num_labels=len(label_cols)


####Generate predictions for the test set

In [77]:
pred_probs = Predict(model, test_input_ids,test_attention_masks, 4,5, device="cuda", batch_size=32)


In [78]:
pred_probs.keys()

dict_keys(['class_pred', 'reg_pred'])

In [79]:
c_pred_probs=pred_probs['class_pred']
c_pred_probs

array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       ...,
       [1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]])

Now we will add the predictions classification labels columns for each original label

In [94]:
label_cols = ['introverted', 'intuitive', 'thinking', 'perceiving']

test["pred_introverted"] = c_pred_probs[:,0]
test["pred_intuitive"] = c_pred_probs[:,1]
test["pred_thinking"] = c_pred_probs[:,2]
test["pred_perceiving"] = c_pred_probs[:,3]


To take a closure look to the classification performance of our model

In [95]:
test.drop(['author', 'body','created_utc',	'agreeableness'	,'openness'	,'conscientiousness'	,'extraversion',	'neuroticism'],axis=1).head()

,introverted,intuitive,thinking,perceiving,pred_introverted,pred_intuitive,pred_thinking,pred_perceiving
320807,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
562195,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
649370,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1023190,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
117790,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0


In [96]:
real_values=test[label_cols]
predicted_values=test[['pred_introverted','pred_intuitive','pred_thinking','pred_perceiving']]

In [97]:
predicted_values.head()

,pred_introverted,pred_intuitive,pred_thinking,pred_perceiving
320807,1.0,1.0,1.0,1.0
562195,1.0,1.0,1.0,1.0
649370,1.0,1.0,1.0,1.0
1023190,0.0,1.0,1.0,1.0
117790,1.0,1.0,1.0,1.0


To evaluate the performance of our classification head we will use different metrics such as f1 score, accuracy, precison etc.. for each class

In [98]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
name_columns=real_values.columns.values
metrics={}
metrics["accuracy_score"]=accuracy_score
metrics["precision_score"]=precision_score
metrics["recall_score"]=recall_score
metrics["f1_score"]=f1_score



In [99]:
test_evaluation_values={}
for column in name_columns:
    test_evaluation_values[str(column)]=[]

In [100]:
for k,v in  metrics.items():
  for column in name_columns:
    test_evaluation_values[str(column)].append(v(real_values[column].values,predicted_values["pred_"+str(column)].values))



In [101]:
d=pd.DataFrame(data=test_evaluation_values,index=list(metrics.keys())).T # F1 score, recall is always better
avg=[]
avg=[d['accuracy_score'].mean(),d['precision_score'].mean(),d['recall_score'].mean(),d['f1_score'].mean()]
d=d.append(dict(zip(d.columns,avg)), ignore_index=True)
d.index=list(name_columns)+['avg']

In [90]:
d

,accuracy_score,precision_score,recall_score,f1_score
introverted,0.648300,0.675554,0.906102,0.774025
intuitive,0.913139,0.913139,1.000000,0.954598
thinking,0.768833,0.768833,1.000000,0.869311
perceiving,0.610992,0.610992,1.000000,0.758529
avg,0.735316,0.742129,0.976525,0.839116


We can see that our model is doing much better comparing to state of the art models baselines on this dataset.

https://paperswithcode.com/dataset/pandora


In [ ]:
#all shaerd saved model
d

,accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
introverted,0.666954,0.668361,0.996568,0.800115,0.498862
intuitive,0.907734,0.907734,1.000000,0.951636,0.500000
thinking,0.763333,0.763333,1.000000,0.865784,0.500000
perceiving,0.563400,0.592649,0.896310,0.713515,0.473200
avg,0.725355,0.733019,0.973219,0.832762,0.493015


#Evaluating the regression head 

like we said our prediction method produce two different predictions. The classification prediction that we evaluated it above and the regression predictions

In [102]:
r_pred_probs=pred_probs['reg_pred']
r_pred_probs

array([[40.47015381, 66.40448761, 41.4925766 , 36.64008331, 35.2992897 ],
       [43.52455521, 67.58790588, 45.7296257 , 38.35994339, 32.94988632],
       [29.68279266, 68.52210999, 24.26068687, 34.07448578, 53.85827255],
       ...,
       [31.70005226, 62.95755386, 30.34681702, 30.55232811, 42.1090889 ],
       [47.86090088, 75.17791748, 46.73184967, 48.37638855, 40.24618149],
       [26.96555519, 61.33392715, 23.00028229, 31.9671936 , 46.79043198]])

Now we will add the predictions regression labels columns for each original label

In [103]:
label_cols = ["agreeableness",	"openness",	"conscientiousness",	"extraversion",	"neuroticism"]

test["pred_agreeableness"] = r_pred_probs[:,0]
test["pred_openness"] = r_pred_probs[:,1]
test["pred_conscientiousness"] = r_pred_probs[:,2]
test["pred_extraversion"] = r_pred_probs[:,3]
test["pred_neuroticism"] = r_pred_probs[:,4]


In [104]:
test.drop(['author','created_utc','introverted',	'intuitive',	'thinking',	'perceiving','pred_introverted',	'pred_intuitive',	'pred_thinking',	'pred_perceiving'],axis=1).head()

,body,agreeableness,openness,conscientiousness,extraversion,neuroticism,pred_agreeableness,pred_openness,pred_conscientiousness,pred_extraversion,pred_neuroticism
320807,Luckily I find that other women are more than ...,98.0,40.0,1.0,1.0,99.0,40.470154,66.404488,41.492577,36.640083,35.299290
562195,[Here](http://www.virtualbattlemap.com/) ya go...,78.0,57.0,38.0,31.0,10.0,43.524555,67.587906,45.729626,38.359943,32.949886
649370,"Washington, UCF, K-State, Colorado, Pitt in no...",3.0,92.0,4.0,97.0,1.0,29.682793,68.522110,24.260687,34.074486,53.858273
1023190,Doot doot,90.0,13.0,20.0,2.0,99.0,23.846178,62.255493,18.226322,33.040768,53.325684
117790,"Hm, so up until a certain point, both old and ...",45.0,69.0,26.0,0.0,28.0,32.988369,59.293369,33.812134,25.924055,34.481518


In [105]:
real_values=test[label_cols]
label_cols = ["agreeableness",	"openness",	"conscientiousness",	"extraversion",	"neuroticism"]

predicted_values=test[['pred_agreeableness','pred_openness','pred_conscientiousness','pred_extraversion','pred_neuroticism']]

Due to the fact that onl the original paper of the Pandora dataset had tried to rpedict the label values of the BIG5 as a regression task we will evaluate our model comparing to the metric that they used duringthe evaluation prosess. In this paper the authors used the person correlation metric to compute the correlation between the original regression labels and the predicted labels and they score a 0.3 correlation. Thats why we will use the person correlation metric to evaluate the performance of our model comparing to the performance of this paper models

In [109]:
#qll shqred
audtorch.metrics.functional.pearsonr(torch.tensor(real_values.values), torch.tensor(predicted_values.values), batch_first=True).mean()


tensor(0.4694, dtype=torch.float64)

we acheivef a 0.469 correlation between the predicted and the original values which overpass the value provided by the state of the art basline for the Pandora datset

To more investigate the perfomance of our model we used different other metrics such as mean square error, r2 score, the mean absolute error etc.. 

In [110]:
from sklearn.metrics import mean_squared_error,max_error,mean_absolute_error,mean_squared_log_error,median_absolute_error,r2_score
name_columns=real_values.columns.values
metrics={}
metrics["mean_squared_error"]=mean_squared_error
metrics["max_error"]=max_error
metrics["mean_absolute_error"]=mean_absolute_error
metrics["mean_squared_log_error"]=mean_squared_log_error
metrics["median_absolute_error"]=median_absolute_error
metrics["r2_score"]=r2_score



In [111]:
test_evaluation_values={}
for column in name_columns:
    test_evaluation_values[str(column)]=[]

In [112]:
for k,v in  metrics.items():
  for column in name_columns:
    if column !='person r correlation':
      test_evaluation_values[str(column)].append(v(real_values[column].values,predicted_values["pred_"+str(column)].values))
    else:
      test_evaluation_values[str(column)].append(v(real_values[[column]].values,predicted_values[["pred_"+str(column)]].values).mean())


In [113]:
d=pd.DataFrame(data=test_evaluation_values,index=list(metrics.keys())).T # F1 score, recall is always better
avg=[]
avg=[d['mean_squared_error'].mean(),d['max_error'].mean(),d['mean_absolute_error'].mean(),d['mean_squared_log_error'].mean(),d['median_absolute_error'].mean(),d['r2_score'].mean()]
d=d.append(dict(zip(d.columns,avg)), ignore_index=True)
d.index=list(name_columns)+['avg']

In [114]:
#saved model shared weights
d

,mean_squared_error,max_error,mean_absolute_error,mean_squared_log_error,median_absolute_error,r2_score
agreeableness,882.814831,79.006084,25.573411,1.792167,23.597282,0.038895
openness,535.316759,83.240517,18.829544,0.276530,17.648758,-0.006352
conscientiousness,948.015190,81.123352,25.685431,1.722717,21.957897,0.065252
extraversion,902.290094,73.727119,26.240673,2.034806,25.509546,0.062167
neuroticism,984.780675,67.711531,27.752496,1.247513,28.652016,0.053905
avg,850.643510,76.961721,24.816311,1.414746,23.473100,0.042773


In [ ]:

import pickle
with open ('doubleheadModel.pickle','wb') as f:
  pickle.dump(model,f)